In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

## Loading GPT-2

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Cargar modelo y tokenizador
model_path = "./gpt2_model_trained_3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

print("Modelo cargado correctamente.")

Modelo cargado correctamente.


## Preparing data

In [12]:
from datasets import load_dataset

data_path = "data_4.txt"
dataset = load_dataset("text", data_files={"train": data_path})

# Ver un ejemplo del dataset
(dataset["train"][:4])

Generating train split: 0 examples [00:00, ? examples/s]

{'text': ['Título: Si algo así como el amor está en el aire',
  'Artista: Mar de Copas',
  'Sentimientos: positivo, alegria, anticipacion',
  '¿como vida encuentras destruido valido vida luego futuro nego mundo gritaba vales futuro nego joya regalo ¿que ver realidad dime ¿quien sabe querer brazos entrego ¿no seria demasiado bien lado ¿cuanto dura ser feliz segundos mas demas caminar vida ley agarrarlo vuelo hoy si asi amor aire final debo adios solo gesto amor saber decirlo mas ¿en momento escribir carta adios solo quiero dia mas quizas dos you might also likedespues sueño tan largo cura sueño sueño mas bellos cada lugar risa mejor risa boca mejor boca beso mejor beso olvido peor olvido ¿a importa ley ¿a importa buen dios nunca supe merecer cosas regalo consuelo fugaz si asi amor aire mano dio quito siempre pude saber devolver ¡y devuelvo mas si asi amor aire si asi amor aire']}

In [13]:
def tokenize_function(examples):
    tokenized_output = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    tokenized_output["labels"] = tokenized_output["input_ids"].copy()
    return tokenized_output

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8124 [00:00<?, ? examples/s]

## Training GPT-2

In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [15]:
from transformers import TrainingArguments, Trainer

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",  # Carpeta donde guardar los modelos entrenados
    run_name="fine_tuning_gpt2_lyrics",
    overwrite_output_dir=True,  # Sobrescribir salida previa
    num_train_epochs=3,  # Número de épocas
    per_device_train_batch_size=4,  # Tamaño del batch
    save_steps=500,  # Guardar cada 500 pasos
    save_total_limit=2,  # Guardar solo los 2 últimos checkpoints
    logging_dir="./logs", # Directorio para logs
    report_to="none",
    logging_steps=1000,  # Intervalo de logs
    do_train=True,  # Habilitar entrenamiento
    do_eval=False,  # Omitir evaluación
    warmup_steps=100,  # Número de pasos de calentamiento
    learning_rate=5e-5,  # Tasa de aprendizaje
    weight_decay=0.01,  # Decaimiento de pesos
)


In [16]:
# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Iniciar el entrenamiento
trainer.train()


Step,Training Loss
1000,0.315800
2000,0.341900
3000,0.275100
4000,0.299700
5000,0.254800
6000,0.283000


TrainOutput(global_step=6093, training_loss=0.29491920007893196, metrics={'train_runtime': 3926.4698, 'train_samples_per_second': 6.207, 'train_steps_per_second': 1.552, 'total_flos': 6368209403904000.0, 'train_loss': 0.29491920007893196, 'epoch': 3.0})

In [17]:
model.save_pretrained("./gpt2_model_trained_4")
tokenizer.save_pretrained("./gpt2_model_trained_4")

('./gpt2_model_trained_4/tokenizer_config.json',
 './gpt2_model_trained_4/special_tokens_map.json',
 './gpt2_model_trained_4/vocab.json',
 './gpt2_model_trained_4/merges.txt',
 './gpt2_model_trained_4/added_tokens.json',
 './gpt2_model_trained_4/tokenizer.json')

In [18]:
# prompt: conectar a drive y guardar mi carpeta gp2 allí

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /content/drive/MyDrive/gpt2_model_trained_4
!cp -r ./gpt2_model_trained_4/ /content/drive/MyDrive/gpt2_model_trained_4

Mounted at /content/drive
